In [1]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
#나라 승인 여부도 실시간 업뎃 할거면 이것까지
#url = 'https://github.com/owid/covid-19-data/blob/master/public/data/vaccinations/locations.csv?raw=true'
#df = pd.read_csv(url)
#df['location'] = df['location'].str.lower()
#Pfizer_country = df[df['vaccines'].str.contains('Pfizer/BioNTech')].drop(['source_website','source_name'],axis=1)
#Moderna_country = df[df['vaccines'].str.contains('Moderna')].drop(['source_website','source_name'],axis=1)
#Pfizer_country['location'] = Pfizer_country['location'].str.lower()
#Moderna_country['location'] = Moderna_country['location'].str.lower()
#Pfizer_country.to_csv('/content/drive/MyDrive/DA팀 /프로젝트/코로나 백신/Pfizer_country.csv')
#Moderna_country.to_csv('/content/drive/MyDrive/DA팀 /프로젝트/코로나 백신/Moderna_country.csv')

In [2]:
#후에 경로변경 해야함
Pfizer_country = pd.read_csv('/content/drive/MyDrive/DA팀 /프로젝트/코로나 백신/Pfizer_country.csv')
Moderna_country = pd.read_csv('/content/drive/MyDrive/DA팀 /프로젝트/코로나 백신/Moderna_country.csv')

In [3]:
pfizer = pd.read_csv('/content/drive/MyDrive/DA팀 /프로젝트/코로나 백신/vaccination_tweets.csv')
moderna = pd.read_csv('/content/drive/MyDrive/DA팀 /프로젝트/코로나 백신/moderna_tweets_0214.csv')

In [4]:
def Preprocessing(file):
  raw_data = file
  raw_data = raw_data.drop(['id','user_created','source'],axis=1)
  data1 = pd.DataFrame(raw_data[['user_name','user_location','user_followers','user_favourites','user_verified','favorites','is_retweet']])
  data2 = pd.DataFrame(raw_data['text'])
  #location
  data1['user_location'] = data1['user_location'].str.lower()
  data_co = data1.copy()
  for x in Pfizer_country['location']:
    for k in range(len(data_co)):
      if x in str(data_co['user_location'][k]):
        data_co['user_location'][k] = 1
  for k in range(len(data_co)):
    if 'london' in str(data_co['user_location'][k]):
      data_co['user_location'][k] = 1
  for k in range(len(data_co)):
    if 'uk' in str(data_co['user_location'][k]):
      data_co['user_location'][k] = 1
  for k in range(len(data_co)):
    if data_co['user_location'][k] != 1  :
      data_co['user_location'][k] = 0
  data1 = data_co
  #dropna
  data1 = data1[data1['is_retweet'].isnull()==False]
  data1 = data1.drop(['is_retweet'],axis=1)
  data1 = data1.reset_index(drop = True)
  #normalization
  #data1['user_followers']=float(data1['user_followers'])/float(max(data1['user_followers']))
  data1['user_favourites']=data1['user_favourites']/max(data1['user_favourites'])
  data1['favorites']=data1['favorites']/max(data1['favorites'])
  return data1, data2
  



In [5]:
#Data2 처리를 위한 함수
def preprocess_tweet_data(data,name):
    # Lowering the case of the words in the sentences
    data[name]=data[name].str.lower()
    # Code to remove the Hashtags from the text
    data[name]=data[name].apply(lambda x:re.sub(r'#','',x))
    # Code to remove the links from the text
    data[name]=data[name].apply(lambda x:re.sub(r"http\S+", "", x))
    # Code to remove the Special characters from the text 
    data[name]=data[name].apply(lambda x:' '.join(re.findall(r'\w+', x)))
    # Code to substitute the multiple spaces with single spaces
    data[name]=data[name].apply(lambda x:re.sub(r'\s+', ' ', x, flags=re.I))
    # Code to remove all the single characters in the text
    data[name]=data[name].apply(lambda x:re.sub(r'\s+[a-zA-Z]\s+', '', x))
    # Remove the twitter handlers
    data[name]=data[name].apply(lambda x:re.sub('@[^\s]+','',x))
def rem_stopwords_tokenize(data,name):

    def getting(sen):
        example_sent = sen

        stop_words = set(stopwords.words('english')) 

        word_tokens = word_tokenize(example_sent) 

        filtered_sentence = [w for w in word_tokens if not w in stop_words] 

        filtered_sentence = [] 

        for w in word_tokens: 
            if w not in stop_words: 
                filtered_sentence.append(w.lower()) 
        return filtered_sentence
    x=[]
    for i in data[name].values:
        x.append(getting(i))
    data[name]=x
# Making a function to lemmatize all the words
lemmatizer = WordNetLemmatizer() 
def lemmatize_all(data,name):
    arr=data[name]
    a=[]
    for i in arr:
        b=[]
        for j in i:
            x=lemmatizer.lemmatize(j,pos='a')
            x=lemmatizer.lemmatize(x)
            b.append(x)
        a.append(b)
    data[name]=a
# Function to make it back into a sentence 
def make_sentences(data,name):
    data[name]=data[name].apply(lambda x:' '.join([i+' ' for i in x]))
    # Removing double spaces if created
    data[name]=data[name].apply(lambda x:re.sub(r'\s+', ' ', x, flags=re.I))
    

In [6]:
#Function to run all above
def preprocess_all(file):
  data1, data2 = Preprocessing(file)
  name = 'text'
  preprocess_tweet_data(data2,name)
  rem_stopwords_tokenize(data2,name)
  lemmatize_all(data2,name)
  make_sentences(data2,name)
  return data1, data2

In [9]:
data1, data2 = preprocess_all(pfizer)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykerne

In [10]:
display(data1)
display(data2)

,user_name,user_location,user_followers,user_favourites,user_verified,favorites
0,Rachel Roh,0,405,0.003512,False,0.000000
1,Albert Fong,0,834,0.000193,False,0.000432
2,eli🇱🇹🇪🇺👌,0,10,0.000168,False,0.000000
3,Charles Adler,1,49165,0.023633,True,0.919654
4,Citizen News Channel,0,152,0.001593,False,0.000000
...,...,...,...,...,...,...
5371,Marc Bailey 💙,1,759,0.005298,False,0.001296
5372,SpaceNews Tech TsunamiHelp,0,427,0.001530,False,0.000000
5373,Lib Dem Lords,0,13959,0.001815,True,0.001296
5374,Scimos.org,0,56,0.000031,False,0.001296


,text
0,folk said daikon paste could treatcytokine sto...
1,world wrong side history year hopefully big va...
2,coronavirus sputnikv astrazeneca pfizerbiontec...
3,fact immutable senator even ethically sturdy e...
4,explain needvaccine borisjohnson matthancock w...
...,...
5371,salmondleeds thanksm counting pfizerbiontech p...
5372,covid19 vaccine best reaction pfizer thanks ja...
5373,lord paulscriven asks reassurance people recei...
5374,part 2 covid vaccine series moderna pfizerbion...
